### Import Packages

In [1]:
from ibm_watson import SpeechToTextV1, NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, EmotionOptions, SentimentOptions
import json
from ipynb.fs.full.audio_recorder import record_and_save_audio
import os

### Record & Save Audio

In [2]:
filename = record_and_save_audio()

Recording...
Finished recording
File Saved as output.wav


### Initiate APIs (Speech to text & NLU)

In [3]:
#All API Credentials embedded in System
speech_api = IAMAuthenticator(os.environ['speech_api_key'])
speech_to_text = SpeechToTextV1(
    authenticator=speech_api)

speech_to_text.set_service_url(os.environ['speech_server_url'])

In [4]:
nlu_api = IAMAuthenticator(os.environ['nlu_api_key'])
nlu = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=nlu_api)

nlu.set_service_url(os.environ['nlu_server_url'])

### Convert Speech to Text

In [5]:
with open(filename,'rb') as audio_file:
    resp = speech_to_text.recognize(
        audio=audio_file,
        content_type='audio/wav').get_result()

In [6]:
transcript_text = ''.join([i['alternatives'][0]['transcript'] for i in resp['results']])

### Use Natural Language Understanding to Parse the text

In [7]:
features_obj=Features(
    entities=EntitiesOptions(emotion=True, sentiment=True, limit=2),
    keywords=KeywordsOptions(emotion=True, sentiment=True, limit=2),
    emotion = EmotionOptions(document=True),
    sentiment= SentimentOptions(document=True)
    )

In [8]:
nlu_resp = nlu.analyze(text=transcript_text,features=features_obj).get_result()

### Prepare Mental Health Profile

In [9]:
class mental_profile:
    overall_emotion = None
    impactful_things = {'positive':[],'negative':[]}
    
    def gen_report(self):
        print('Overall Emotion\t\t\t\t:',self.overall_emotion,end='\n\n\n')
        print('THINGS GOING ON IN PERSON\'s MIND\t\t')
        print('Positive\t\t\t\t:',' | '.join(self.impactful_things['positive']),end='\n\n')
        print('Negative\t\t\t\t:',' | '.join(self.impactful_things['negative']),end='\n\n')
    
    def gen_parameters(self,nlu_response):
        self.overall_emotion = nlu_response['sentiment']['document']['label']
        self.impactful_things['positive'] = [i['text'] for i in nlu_response['keywords'] if i['sentiment']['label']=='positive']
        self.impactful_things['negative'] = [i['text'] for i in nlu_response['keywords'] if i['sentiment']['label']=='negative']

In [10]:
new_pers=mental_profile()
new_pers.gen_parameters(nlu_resp)
print('\n','*'*120,sep='');print('*'*120,end='\n\n')
new_pers.gen_report()
print('\n','*'*120);print('*'*120)


************************************************************************************************************************
************************************************************************************************************************

Overall Emotion				: negative


THINGS GOING ON IN PERSON's MIND		
Positive				: independent

Negative				: boyfriend | jealous | family | depressed


 ************************************************************************************************************************
************************************************************************************************************************
